<a href="https://colab.research.google.com/github/MinuraPunchihewa/Authorship-Identification-Language-Modelling/blob/main/word-language-model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install textract

     |████████████████████████████████| 102kB 5.8MB/s 
     |████████████████████████████████| 8.9MB 15.1MB/s 
     |████████████████████████████████| 112kB 57.7MB/s 
     |████████████████████████████████| 32.8MB 86kB/s 
     |████████████████████████████████| 5.6MB 56.3MB/s 
     |████████████████████████████████| 112kB 61.5MB/s 
     |████████████████████████████████| 51kB 7.9MB/s 
     |████████████████████████████████| 153kB 57.9MB/s 
     |████████████████████████████████| 13.7MB 239kB/s 
     |████████████████████████████████| 112kB 63.9MB/s 
     |████████████████████████████████| 81kB 11.1MB/s 
  Created wheel for python-pptx: filename=python_pptx-0.6.18-cp36-none-any.whl size=275707 sha256=363a30faffb80398f7c71e868f8f7be0598b9e0511a8db4b79004a5067236278
  Stored in directory: /root/.cache/pip/wheels/1f/1f/2c/29acca422b420a0b5210bd2cd7e9669804520d602d2462f20b
  Created wheel for EbookLib: filename=EbookLib-0.17.1-cp36-none-any.whl size=38163 sha256=1a104d52d4d6f644df1f0763b28d

In [ ]:
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import textract

import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
dinesh_asanka = textract.process('/content/drive/My Drive/Dinesh Asanka Train.docx')

In [ ]:
da_sentences = nltk.tokenize.sent_tokenize(dinesh_asanka.decode())

In [ ]:
len(da_sentences)

1419

In [ ]:
tokenizer = Tokenizer(oov_token = '<OOV>', filters='\t\n')
tokenizer.fit_on_texts(da_sentences)

In [ ]:
word_index = tokenizer.word_index
total_words = len(tokenizer.word_index) + 1

In [ ]:
da_sequences = tokenizer.texts_to_sequences(da_sentences)

In [ ]:
max_len = len(max(da_sequences, key=len))

In [ ]:
input_sequences = []
for line in da_sentences:
  token_list = tokenizer.texts_to_sequences([line])[0]
  for i in range(0, len(token_list)):
    n_gram_sequence = token_list[:i+1]
    input_sequences.append(n_gram_sequence)

In [ ]:
padded_sequences = pad_sequences(input_sequences, maxlen = max_len, padding = 'pre')

In [ ]:
inputs = padded_sequences[:, :-1]
labels = padded_sequences[:,-1]

In [ ]:
one_hot_encoded_labels = tf.keras.utils.to_categorical(labels, num_classes = total_words)

In [ ]:
tf.keras.optimizers.Adam(learning_rate=0.01)
model = tf.keras.Sequential([
                             tf.keras.layers.Embedding(total_words, 64, input_length=inputs.shape[1]),
                             tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(100, return_sequences = True)),
                             tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(100)),
                             tf.keras.layers.Dense(100, activation = 'relu'),
                             tf.keras.layers.Dense(total_words, activation = 'softmax')
])

In [ ]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [ ]:
num_epochs = 100
model.fit(inputs, one_hot_encoded_labels, epochs = num_epochs, verbose = 2)

Epoch 1/100
725/725 - 137s - loss: 8.2436 - accuracy: 0.0112
Epoch 2/100
725/725 - 137s - loss: 7.8496 - accuracy: 0.0103
Epoch 3/100
725/725 - 137s - loss: 7.6451 - accuracy: 0.0147
Epoch 4/100
725/725 - 137s - loss: 7.4533 - accuracy: 0.0197
Epoch 5/100
725/725 - 137s - loss: 7.2703 - accuracy: 0.0226
Epoch 6/100
725/725 - 137s - loss: 7.1103 - accuracy: 0.0281
Epoch 7/100
725/725 - 137s - loss: 6.9520 - accuracy: 0.0344
Epoch 8/100
725/725 - 137s - loss: 6.8008 - accuracy: 0.0358
Epoch 9/100
725/725 - 137s - loss: 6.6501 - accuracy: 0.0394
Epoch 10/100
725/725 - 137s - loss: 6.5029 - accuracy: 0.0442
Epoch 11/100
725/725 - 137s - loss: 6.3580 - accuracy: 0.0472
Epoch 12/100
725/725 - 137s - loss: 6.2029 - accuracy: 0.0525
Epoch 13/100
725/725 - 137s - loss: 6.0511 - accuracy: 0.0572
Epoch 14/100
725/725 - 137s - loss: 5.9014 - accuracy: 0.0644
Epoch 15/100
725/725 - 138s - loss: 5.7480 - accuracy: 0.0710
Epoch 16/100
725/725 - 137s - loss: 5.5993 - accuracy: 0.0781
Epoch 17/100
725/

In [ ]:
def prepare_sentences(sentences):
  input_sequences = []
  for line in sentences:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(0, len(token_list)):
      n_gram_sequence = token_list[:i+1]
      input_sequences.append(n_gram_sequence)

      padded_sequences = pad_sequences(input_sequences, maxlen = max_len, padding = 'pre')

      inputs = padded_sequences[:, :-1]
      labels = padded_sequences[:,-1]

      one_hot_encoded_labels = tf.keras.utils.to_categorical(labels, num_classes = total_words)
  return inputs, one_hot_encoded_labels

In [ ]:
aa_data = ['බෝරාවරුන් ඔවුන්ගේ වර්තමාන ආධ්‍යාත්මික නායකයා වූ 53 වන දායිවරයා, සායිද්නා මුෆාදල් ඉතා ගෞරවයෙන් සහ ආදරයෙන් අගය කරනවා.']

In [ ]:
aa_inputs, aa_labels = prepare_sentences(aa_data)

In [ ]:
aa_probabilities = model.predict(aa_inputs)

In [ ]:
for i in range(0, aa_probabilities.shape[0] - 1):
  print(str(i) + " " + str(np.sum(aa_probabilities[i] * aa_labels[i])))

In [ ]:
lk_data = ['මෙසේ වෘත්තීය විෂයයන් ඔස්සේ කර්මාන්ත පුහුණුවක් ලබා ගැනීමෙන් පසුව රැකියාවකට පිවිසීමට හෝ ස්වයං රැකියාවක නිරත වීමට ශිෂ්‍යන්ට හැකියාව ලැබෙනවා.']

In [ ]:
lk_inputs, lk_labels = prepare_sentences(lk_data)

In [ ]:
lk_probabilities = model.predict(lk_inputs)

In [ ]:
for i in range(0, lk_probabilities.shape[0] - 1):
  print(str(i) + " " + str(np.sum(lk_probabilities[i] * lk_labels[i])))

In [ ]:
da_data = ['මේ චෝදනා රේගන් ජනාධිපතිවරයා මුළුමනින්ම ප්‍රතික්ෂේප කළත් මාධ්‍යයවලින් ආ පීඩනය නිසාම ටවර් කොමිසම පත්කළා.']

In [ ]:
da_inputs, da_labels = prepare_sentences(da_data)

In [ ]:
da_probabilities = model.predict(da_inputs)

In [ ]:
for i in range(0, da_probabilities.shape[0] - 1):
  print(str(i) + " " + str(np.sum(da_probabilities[i] * da_labels[i])))

In [ ]:
def calculate_probability(probabilities, labels):
  log_prob_sentence = 0
  for i in range(0, probabilities.shape[0] - 1):
    log_prob_sentence += np.log(np.sum(probabilities[i] * labels[i]))

  sentence_probability = np.exp(log_prob_sentence)
  return log_prob_sentence